In [1]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')
dfHeart_discretize

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,Target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [2]:
dfHeart_discretize.groupby('Target').describe()

age                                                       sex  \
        count       mean       std   min   25%   50%   75%   max  count   
Target                                                                    
0       138.0  56.601449  7.962082  35.0  52.0  58.0  62.0  77.0  138.0   
1       165.0  52.496970  9.550651  29.0  44.0  52.0  59.0  76.0  165.0   

                  ...  caa       thall                                     \
            mean  ...  75%  max  count      mean       std  min  25%  50%   
Target            ...                                                       
0       0.826087  ...  2.0  4.0  138.0  2.543478  0.684762  0.0  2.0  3.0   
1       0.563636  ...  0.0  4.0  165.0  2.121212  0.465752  0.0  2.0  2.0   

                  
        75%  max  
Target            
0       3.0  3.0  
1       2.0  3.0  

[2 rows x 104 columns]

In [3]:
from sklearn.ensemble import RandomForestClassifier
from slice_finder import SliceFinder
import warnings
warnings.filterwarnings('ignore')

# View regions

In [4]:
with open('./Slices/Heart_test_1_RF_5_10.p', 'rb') as file:
    data = pickle.load(file)
    
    data = sorted(data, key=lambda d: d.size, reverse=True)
    for d in data:
        print(f'Description of the region:  {d.filters}')
        print(f'Region instances: {d.data_idx}')
        print(f'Region size:    {d.size}')
        print(f'Efect size: {d.effect_size}')
        print(f'Log Loss:    {d.metric}')
        print('\n')

Description of the region:  {'cp': [[3]], 'sex': [[1]]}
Region instances: Index([  0,  13,  24,  34,  58,  62,  83, 100, 101, 106, 117, 152, 222, 228,
       254, 259, 271, 286, 299],
      dtype='int64')
Region size:    19
Efect size: 0.42738790588141745
Log Loss:    0.4678265571574056


Description of the region:  {'chol': [[230.0, 240.0]], 'slp': [[1]]}
Region instances: Index([20, 52, 67, 106, 107, 183, 188, 196, 206, 224, 259, 271, 272, 302], dtype='int64')
Region size:    14
Efect size: 0.4675843019481437
Log Loss:    0.4904088160208883


Description of the region:  {'cp': [[3]], 'slp': [[1]]}
Region instances: Index([13, 62, 83, 106, 117, 152, 222, 228, 259, 271, 299], dtype='int64')
Region size:    11
Efect size: 0.4085307732769338
Log Loss:    0.46073235375253246


Description of the region:  {'age': [[64]]}
Region instances: Index([13, 110, 112, 120, 139, 152, 177, 229, 233, 242], dtype='int64')
Region size:    10
Efect size: 0.4835642850489318
Log Loss:    0.5329037314643092

# Check metrics

In [5]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X, y, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Time to train the model: 3.283867120742798 seconds
Accuracy: 0.81 (+/- 0.07)
Recall: 0.84 (+/- 0.13)
F1 Score: 0.82 (+/- 0.07)
Log-loss: 0.41 (+/- 0.09)


In [6]:
import pandas as pd
from utils import *
from utils_datasets import *

df_census = import_datasets('heart')
dfCensus_discretize, X, y, encoders = encoder(df_census, 'heart')

df_lenght = len(dfCensus_discretize)
    
smote = SMOTE(sampling_strategy=0.9, random_state=42)
    
y_slice = dfCensus_discretize['Target']
X_slice = dfCensus_discretize.drop(columns='Target')
    
start_time = time.time()
X_slice_smote, y_slice_smote = smote.fit_resample(X_slice, y_slice)
region_SMOTE = (pd.concat([pd.DataFrame(X_slice_smote, columns=X_slice.columns), pd.Series(y_slice_smote, name='Target')], axis=1))
end_time = time.time()
execution_time = end_time - start_time
    
instance_add = len(region_SMOTE) - df_lenght
    
y_SMOTE_data = region_SMOTE['Target']
X_SMOTE_data = region_SMOTE.drop(columns='Target')
    
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model


print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 10 with time of: 0.021300315856933594 seconds to apply SMOTE
Time to train the model: 3.283970355987549 seconds
Accuracy: 0.82 (+/- 0.06)
Recall: 0.85 (+/- 0.11)
F1 Score: 0.83 (+/- 0.06)
Log-loss: 0.40 (+/- 0.05)


# SMOTE in regions

In [1]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = 'oldpeak==1 and restecg==1'

instance_add, execution_time, X_SMOTE_data, y_SMOTE_data = apply_SMOTE_region(dfHeart_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 3 with time of: 0.5109176635742188 seconds to apply SMOTE
Time to train the model: 2.781543254852295 seconds
Accuracy: 0.83 (+/- 0.04)
Recall: 0.89 (+/- 0.06)
F1 Score: 0.85 (+/- 0.03)
Log-loss: 0.39 (+/- 0.03)


In [2]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = 'cp==3 and thall==2'

instance_add, execution_time, X_SMOTE_data, y_SMOTE_data = apply_SMOTE_region(dfHeart_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 5 with time of: 0.01628398895263672 seconds to apply SMOTE
Time to train the model: 3.0196681022644043 seconds
Accuracy: 0.83 (+/- 0.07)
Recall: 0.87 (+/- 0.10)
F1 Score: 0.84 (+/- 0.07)
Log-loss: 0.39 (+/- 0.06)


In [3]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = 'chol>=230.0 and chol<=240.0 and slp==1'

instance_add, execution_time, X_SMOTE_data, y_SMOTE_data = apply_SMOTE_region(dfHeart_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 3 with time of: 0.0127716064453125 seconds to apply SMOTE
Time to train the model: 3.3355414867401123 seconds
Accuracy: 0.83 (+/- 0.06)
Recall: 0.88 (+/- 0.07)
F1 Score: 0.85 (+/- 0.05)
Log-loss: 0.40 (+/- 0.09)


In [4]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = 'cp==3 and thall==3'

instance_add, execution_time, X_SMOTE_data, y_SMOTE_data = apply_SMOTE_region(dfHeart_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 2 with time of: 0.019996166229248047 seconds to apply SMOTE
Time to train the model: 0.514711856842041 seconds
Accuracy: 0.81 (+/- 0.08)
Recall: 0.83 (+/- 0.07)
F1 Score: 0.82 (+/- 0.07)
Log-loss: 0.43 (+/- 0.09)


In [5]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = 'thalachh==152'

instance_add, execution_time, X_SMOTE_data, y_SMOTE_data = apply_SMOTE_region(dfHeart_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 4 with time of: 0.015688657760620117 seconds to apply SMOTE
Time to train the model: 2.4911458492279053 seconds
Accuracy: 0.81 (+/- 0.05)
Recall: 0.85 (+/- 0.09)
F1 Score: 0.82 (+/- 0.06)
Log-loss: 0.42 (+/- 0.09)


In [6]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = 'age==57 and slp==2'

instance_add, execution_time, X_SMOTE_data, y_SMOTE_data = apply_SMOTE_region(dfHeart_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 3 with time of: 0.019299983978271484 seconds to apply SMOTE
Time to train the model: 0.5514569282531738 seconds
Accuracy: 0.82 (+/- 0.05)
Recall: 0.87 (+/- 0.08)
F1 Score: 0.84 (+/- 0.05)
Log-loss: 0.43 (+/- 0.07)


In [7]:
import pandas as pd
from utils import *
from utils_datasets import *

df_census = import_datasets('heart')
dfCensus_discretize, X, y, encoders = encoder(df_census, 'heart')

region = ['cp==3 and thall==3',
          'cp==3 and thall==2']

instances_add, X_SMOTE_data, y_SMOTE_data = region_smote(dfCensus_discretize, region)

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Instances added: {instance_add} with time of: {execution_time} seconds to apply SMOTE")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Instances added: 3 with time of: 0.019299983978271484 seconds to apply SMOTE
Time to train the model: 0.5524492263793945 seconds
Accuracy: 0.82 (+/- 0.08)
Recall: 0.87 (+/- 0.09)
F1 Score: 0.84 (+/- 0.07)
Log-loss: 0.42 (+/- 0.11)


# Case study

In [8]:
with open('./Slices/Heart_otimizado.p', 'rb') as file:
    data = pickle.load(file)
    
    name = []
    aux_1 = []
    aux_2 = []
    
    data = sorted(data, key=lambda d: d.size, reverse=True)
    for d in data:
        print ('\n=====================\nSlice description:')
        for k, v in list(d.filters.items()):
            values = ''
            if k in encoders:
                le = encoders[k]
                for v_ in v:
                    values += '%s '%(le.inverse_transform(v_)[0])
            else:
                for v_ in sorted(v, key=lambda x: x[0]):
                    if len(v_) > 1:
                        values += '%s ~ %s'%(v_[0], v_[1])
                    else:
                        values += '%s '%(v_[0])
            print ('%s:%s'%(k, values))
        print(f'Description of the region:  {d.filters}')
        print(f'Region instances: {d.data_idx}')
        print(f'Region size:    {d.size}')
        print(f'Efect size: {d.effect_size}')
        print(f'Log Loss:    {d.metric}')
        print('\n')
        
        if d.size > 10:
            aux_1.append(d.size)
            aux_2.append(d.effect_size)


Slice description:
oldpeak:1.0 
restecg:1 
Description of the region:  {'oldpeak': [[1.0]], 'restecg': [[1]]}
Region instances: Index([23, 86, 171, 260, 272, 275, 281], dtype='int64')
Region size:    7
Efect size: 0.42355189929305403
Log Loss:    0.2395213109934631



Slice description:
age:64 
restecg:1 
Description of the region:  {'age': [[64]], 'restecg': [[1]]}
Region instances: Index([110, 112, 120, 139, 177, 229], dtype='int64')
Region size:    6
Efect size: 0.5721909740708442
Log Loss:    0.29638876984520807



Slice description:
oldpeak:0.2 
thall:3 
Description of the region:  {'oldpeak': [[0.2]], 'thall': [[3]]}
Region instances: Index([111, 112, 139, 197, 228, 298], dtype='int64')
Region size:    6
Efect size: 0.4405615621736584
Log Loss:    0.24590707144858703



Slice description:
age:64 
thall:2 
Description of the region:  {'age': [[64]], 'thall': [[2]]}
Region instances: Index([13, 110, 120, 177, 233], dtype='int64')
Region size:    5
Efect size: 0.4207648302002951
Lo

In [9]:
dicionario = {}

for d in data:
    if d.size > 1:
        dicionario[f'{d.filters}'] = []
        for i in d.data_idx:
            dicionario[f'{d.filters}'].append(i)

In [10]:
from collections import Counter
import ast

feature_cont = Counter()

for feature in dicionario.keys():
    key_dict = ast.literal_eval(feature)
    for feature in key_dict.keys():
        feature_cont[feature] += 1

In [11]:
print(feature_cont)

Counter({'thalachh': 32, 'age': 30, 'oldpeak': 25, 'cp': 21, 'chol': 19, 'trtbps': 14, 'slp': 13, 'caa': 12, 'sex': 8, 'thall': 6, 'restecg': 5, 'exng': 5})


In [13]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

In [46]:
dfHeart_discretize.groupby('age').describe()

sex                                                   cp            ...  \
    count      mean       std  min   25%  50%   75%  max count      mean  ...   
age                                                                       ...   
29    1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0   1.0  1.000000  ...   
34    2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0   2.0  2.000000  ...   
35    4.0  0.750000  0.500000  0.0  0.75  1.0  1.00  1.0   4.0  0.250000  ...   
37    2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0   2.0  2.000000  ...   
38    3.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0   3.0  2.333333  ...   
39    4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0   4.0  1.500000  ...   
40    3.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0   3.0  1.000000  ...   
41   10.0  0.600000  0.516398  0.0  0.00  1.0  1.00  1.0  10.0  1.200000  ...   
42    8.0  0.750000  0.462910  0.0  0.75  1.0  1.00  1.0   8.0  1.250000  ...   
43    8.0  0.750000  0.462910  0.0  0.75  1.0  1.00  1.0   8.0  0.500000  ...   
44   11.0  0.818182  0.404520  0.0  1.00  1.0  1.00  1.0  11.0  1.181818  ...   
45    8.0  0.625000  0.517549  0.0  0.00  1.0  1.00  1.0   8.0  0.750000  ...   
46    7.0  0.571429  0.534522  0.0  0.00  1.0  1.00  1.0   7.0  0.857143  ...   
47    5.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0   5.0  1.200000  ...   
48    7.0  0.857143  0.377964  0.0  1.00  1.0  1.00  1.0   7.0  0.857143  ...   
49    5.0  0.600000  0.547723  0.0  0.00  1.0  1.00  1.0   5.0  1.200000  ...   
50    7.0  0.571429  0.534522  0.0  0.00  1.0  1.00  1.0   7.0  1.000000  ...   
51   12.0  0.666667  0.492366  0.0  0.00  1.0  1.00  1.0  12.0  1.416667  ...   
52   13.0  0.923077  0.277350  0.0  1.00  1.0  1.00  1.0  13.0  1.153846  ...   
53    8.0  0.625000  0.517549  0.0  0.00  1.0  1.00  1.0   8.0  0.750000  ...   
54   16.0  0.687500  0.478714  0.0  0.00  1.0  1.00  1.0  16.0  1.062500  ...   
55    8.0  0.500000  0.534522  0.0  0.00  0.5  1.00  1.0   8.0  0.375000  ...   
56   11.0  0.727273  0.467099  0.0  0.50  1.0  1.00  1.0  11.0  0.818182  ...   
57   17.0  0.764706  0.437237  0.0  1.00  1.0  1.00  1.0  17.0  0.529412  ...   
58   19.0  0.684211  0.477567  0.0  0.00  1.0  1.00  1.0  19.0  0.842105  ...   
59   14.0  0.928571  0.267261  0.0  1.00  1.0  1.00  1.0  14.0  1.214286  ...   
60   11.0  0.636364  0.504525  0.0  0.00  1.0  1.00  1.0  11.0  0.818182  ...   
61    8.0  0.750000  0.462910  0.0  0.75  1.0  1.00  1.0   8.0  0.625000  ...   
62   11.0  0.363636  0.504525  0.0  0.00  0.0  1.00  1.0  11.0  0.545455  ...   
63    9.0  0.444444  0.527046  0.0  0.00  0.0  1.00  1.0   9.0  0.666667  ...   
64   10.0  0.700000  0.483046  0.0  0.25  1.0  1.00  1.0  10.0  1.200000  ...   
65    8.0  0.500000  0.534522  0.0  0.00  0.5  1.00  1.0   8.0  1.125000  ...   
66    7.0  0.571429  0.534522  0.0  0.00  1.0  1.00  1.0   7.0  0.857143  ...   
67    9.0  0.666667  0.500000  0.0  0.00  1.0  1.00  1.0   9.0  0.666667  ...   
68    4.0  0.750000  0.500000  0.0  0.75  1.0  1.00  1.0   4.0  1.500000  ...   
69    3.0  0.666667  0.577350  0.0  0.50  1.0  1.00  1.0   3.0  2.666667  ...   
70    4.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0   4.0  0.750000  ...   
71    3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   3.0  1.000000  ...   
74    1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0   1.0  1.000000  ...   
76    1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0   1.0  2.000000  ...   
77    1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0   1.0  0.000000  ...   

    thall      Target                                                 
      75%  max  count      mean       std  min   25%  50%   75%  max  
age                                                                   
29   2.00  2.0    1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0  
34   2.00  2.0    2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0  
35   3.00  3.0    4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0  
37   2.00  2.0 

In [ ]:
dfHeart_discretize.groupby('cp').describe()

age                                                         sex  \
    count       mean        std   min    25%   50%   75%   max  count   
cp                                                                      
0   143.0  55.692308   8.312752  35.0  51.00  57.0  62.0  77.0  143.0   
1    50.0  51.360000   9.512098  29.0  44.25  52.0  57.0  74.0   50.0   
2    87.0  53.517241   9.420243  37.0  46.50  53.0  60.0  76.0   87.0   
3    23.0  55.869565  10.033147  34.0  51.50  59.0  63.5  69.0   23.0   

              ... thall      Target                                          \
        mean  ...   75%  max  count      mean       std  min  25%  50%  75%   
cp            ...                                                             
0   0.727273  ...   3.0  3.0  143.0  0.272727  0.446927  0.0  0.0  0.0  1.0   
1   0.640000  ...   2.0  3.0   50.0  0.820000  0.388088  0.0  1.0  1.0  1.0   
2   0.597701  ...   2.5  3.0   87.0  0.793103  0.407429  0.0  1.0  1.0  1.0   
3   0.826087  ...   3.0  3.0   23.0  0.695652  0.470472  0.0  0.0  1.0  1.0   

         
    max  
cp       
0   1.0  
1   1.0  
2   1.0  
3   1.0  

[4 rows x 104 columns]

In [ ]:
dfHeart_discretize.groupby('oldpeak').describe()

age                                                         sex  \
        count       mean        std   min    25%   50%    75%   max count   
oldpeak                                                                     
0.0      99.0  50.757576   9.328572  29.0  44.00  51.0  58.00  77.0  99.0   
0.1       7.0  55.285714   9.586697  43.0  49.50  52.0  62.00  69.0   7.0   
0.2      12.0  56.500000   9.605869  43.0  48.00  57.0  64.00  74.0  12.0   
0.3       3.0  56.000000  11.532563  44.0  50.50  57.0  62.00  67.0   3.0   
0.4       9.0  58.333333   8.062258  44.0  54.00  57.0  65.00  71.0   9.0   
0.5       5.0  52.800000   4.086563  48.0  51.00  52.0  54.00  59.0   5.0   
0.6      14.0  53.357143   7.088530  42.0  49.25  53.5  57.75  65.0  14.0   
0.7       1.0  34.000000        NaN  34.0  34.00  34.0  34.00  34.0   1.0   
0.8      13.0  55.461538   8.809232  42.0  49.00  56.0  65.00  67.0  13.0   
0.9       3.0  59.000000   8.544004  50.0  55.00  60.0  63.50  67.0   3.0   
1.0      14.0  58.500000   6.813448  47.0  53.25  58.5  64.00  68.0  14.0   
1.1       2.0  63.000000  18.384776  50.0  56.50  63.0  69.50  76.0   2.0   
1.2      17.0  53.588235   6.423189  39.0  51.00  55.0  57.00  62.0  17.0   
1.3       1.0  56.000000        NaN  56.0  56.00  56.0  56.00  56.0   1.0   
1.4      13.0  53.384615   9.987813  35.0  46.00  55.0  62.00  65.0  13.0   
1.5       5.0  57.200000  10.639549  43.0  51.00  57.0  67.00  68.0   5.0   
1.6      11.0  56.545455  10.033580  35.0  52.50  56.0  63.00  71.0  11.0   
1.8      10.0  59.300000   8.551153  42.0  59.00  62.5  63.75  69.0  10.0   
1.9       5.0  55.600000   7.569676  43.0  56.00  56.0  61.00  62.0   5.0   
2.0       9.0  54.777778  10.170764  40.0  49.00  55.0  64.00  69.0   9.0   
2.1       1.0  56.000000        NaN  56.0  56.00  56.0  56.00  56.0   1.0   
2.2       4.0  58.750000   4.112988  54.0  57.00  58.5  60.25  64.0   4.0   
2.3       2.0  64.500000   2.121320  63.0  63.75  64.5  65.25  66.0   2.0   
2.4       3.0  60.333333   9.504385  51.0  55.50  60.0  65.00  70.0   3.0   
2.5       2.0  50.500000  10.606602  43.0  46.75  50.5  54.25  58.0   2.0   
2.6       6.0  62.333333   7.118052  50.0  60.25  63.5  66.75  70.0   6.0   
2.8       6.0  56.833333   7.222650  44.0  55.00  59.0  60.00  65.0   6.0   
2.9       1.0  70.000000        NaN  70.0  70.00  70.0  70.00  70.0   1.0   
3.0       5.0  52.600000   7.956130  43.0  45.00  57.0  58.00  60.0   5.0   
3.1       1.0  53.000000        NaN  53.0  53.00  53.0  53.00  53.0   1.0   
3.2       2.0  56.000000   2.828427  54.0  55.00  56.0  57.00  58.0   2.0   
3.4       3.0  60.666667   6.658328  55.0  57.00  59.0  63.50  68.0   3.0   
3.5       1.0  37.000000        NaN  37.0  37.00  37.0  37.00  37.0   1.0   
3.6       4.0  57.500000   7.681146  46.0  57.25  61.0  61.25  62.0   4.0   
3.8       1.0  38.000000        NaN  38.0  38.00  38.0  38.00  38.0   1.0   
4.0       3.0  60.666667   4.041452  56.0  59.50  63.0  63.00  63.0   3.0   
4.2       2.0  55.000000   5.656854  51.0  53.00  55.0  57.00  59.0   2.0   
4.4       1.0  58.000000        NaN  58.0  58.00  58.0  58.00  58.0   1.0   
5.6       1.0  55.000000        NaN  55.0  55.00  55.0  55.00  55.0   1.0   
6.2       1.0  62.000000        NaN  62.0  62.00  62.0  62.00  62.0   1.0   

                   ... thall      Target                                      \
             mean  ...   75%  max  count      mean       std  min   25%  50%   
oldpeak            ...                                                         
0.0      0.646465  ...  2.00  3.0   99.0  0.747475  0.436672  0.0  0.50  1.0   
0.1      0.857143  ...  3.00  3.0    7.0  0.571429  0.534522  0.0  0.00  1.0   
0.2      0.500000  ...  3.00  3.0   12.0  0.750000  0.452267  0.0  0.75  1.0   
0.3      0.333333  ...  2.50  3.0    3.0  0.666667  0.577350  0.0  0.50  1.0   
0.4      0.777778  ...  3.00  3.0    9.0  0.888889  0.333333  0.0  1.00  1.0   
0.5      0.800000  ...  3.00  3.0    5.0  0.800000  0.4472

In [ ]:
dfHeart_discretize.groupby('trtbps').describe()

age                                                         sex  \
       count       mean        std   min    25%   50%    75%   max count   
trtbps                                                                     
94       2.0  45.000000   8.485281  39.0  42.00  45.0  48.00  51.0   2.0   
100      4.0  58.500000   6.557439  51.0  56.25  58.0  60.25  67.0   4.0   
101      1.0  46.000000        NaN  46.0  46.00  46.0  46.00  46.0   1.0   
102      2.0  51.000000  12.727922  42.0  46.50  51.0  55.50  60.0   2.0   
104      1.0  45.000000        NaN  45.0  45.00  45.0  45.00  45.0   1.0   
105      3.0  48.333333   8.736895  41.0  43.50  46.0  52.00  58.0   3.0   
106      1.0  67.000000        NaN  67.0  67.00  67.0  67.00  67.0   1.0   
108      6.0  52.333333   6.592926  44.0  48.25  53.0  54.00  63.0   6.0   
110     19.0  51.842105   8.852264  40.0  44.50  51.0  57.00  71.0  19.0   
112      9.0  51.666667  11.022704  41.0  44.00  47.0  58.00  71.0   9.0   
114      1.0  58.000000        NaN  58.0  58.00  58.0  58.00  58.0   1.0   
115      3.0  51.666667  13.316656  43.0  44.00  45.0  56.00  67.0   3.0   
117      1.0  60.000000        NaN  60.0  60.00  60.0  60.00  60.0   1.0   
118      7.0  45.714286  12.037639  34.0  36.50  44.0  50.50  68.0   7.0   
120     37.0  52.945946  10.118591  35.0  44.00  54.0  61.00  74.0  37.0   
122      4.0  45.000000   8.041559  35.0  41.00  45.5  49.50  54.0   4.0   
123      1.0  53.000000        NaN  53.0  53.00  53.0  53.00  53.0   1.0   
124      6.0  55.333333   6.562520  48.0  49.50  55.5  60.75  63.0   6.0   
125     11.0  58.909091   7.917759  51.0  53.00  58.0  62.00  77.0  11.0   
126      3.0  45.000000  12.489996  35.0  38.00  41.0  50.00  59.0   3.0   
128     12.0  55.416667   5.089353  45.0  52.00  56.0  58.00  64.0  12.0   
129      1.0  50.000000        NaN  50.0  50.00  50.0  50.00  50.0   1.0   
130     36.0  52.111111   8.801876  29.0  46.50  53.0  58.50  70.0  36.0   
132      8.0  52.625000   6.069538  43.0  51.25  55.0  56.25  58.0   8.0   
134      5.0  55.400000   4.929503  49.0  52.00  56.0  59.00  61.0   5.0   
135      6.0  56.166667   8.588752  41.0  54.25  57.0  62.00  65.0   6.0   
136      3.0  50.666667   8.082904  42.0  47.00  52.0  55.00  58.0   3.0   
138     13.0  49.230769  10.248202  35.0  39.00  47.0  59.00  65.0  13.0   
140     32.0  56.593750   8.654141  39.0  51.00  57.5  62.25  76.0  32.0   
142      3.0  48.000000   4.358899  45.0  45.50  46.0  49.50  53.0   3.0   
144      2.0  59.000000  12.727922  50.0  54.50  59.0  63.50  68.0   2.0   
145      5.0  63.600000   3.911521  60.0  61.00  63.0  64.00  70.0   5.0   
146      2.0  62.000000   5.656854  58.0  60.00  62.0  64.00  66.0   2.0   
148      2.0  51.500000  13.435029  42.0  46.75  51.5  56.25  61.0   2.0   
150     17.0  57.411765   6.215493  43.0  57.00  58.0  61.00  66.0  17.0   
152      5.0  56.600000  11.326959  40.0  52.00  57.0  67.00  67.0   5.0   
154      1.0  57.000000        NaN  57.0  57.00  57.0  57.00  57.0   1.0   
155      1.0  65.000000        NaN  65.0  65.00  65.0  65.00  65.0   1.0   
156      1.0  70.000000        NaN  70.0  70.00  70.0  70.00  70.0   1.0   
160     11.0  64.000000   5.813777  54.0  60.50  66.0  68.00  71.0  11.0   
164      1.0  59.000000        NaN  59.0  59.00  59.0  59.00  59.0   1.0   
165      1.0  57.000000        NaN  57.0  57.00  57.0  57.00  57.0   1.0   
170      4.0  60.000000   2.708013  58.0  58.75  59.0  60.25  64.0   4.0   
172      1.0  52.000000        NaN  52.0  52.00  52.0  52.00  52.0   1.0   
174      1.0  59.000000        NaN  59.0  59.00  59.0  59.00  59.0   1.0   
178      2.0  62.500000   4.949747  59.0  60.75  62.5  64.25  66.0   2.0   
180      3.0  62.333333   6.658328  55.0  59.50  64.0  66.00  68.0   3.0   
192      1.0  54.000000        NaN  54.0  54.00  54.0  54.00  54.0   1.0   
200      1.0  56.000000        NaN  56.0  56.00  56.0  56.00  56.0   1.0   

                  ... thall      Target                 

In [ ]:
dfHeart_discretize.groupby('thalachh').describe()

age                                                 sex       ...  \
         count  mean       std   min   25%   50%   75%   max count mean  ...   
thalachh                                                                 ...   
71         1.0  67.0       NaN  67.0  67.0  67.0  67.0  67.0   1.0  1.0  ...   
88         1.0  57.0       NaN  57.0  57.0  57.0  57.0  57.0   1.0  1.0  ...   
90         1.0  59.0       NaN  59.0  59.0  59.0  59.0  59.0   1.0  1.0  ...   
95         1.0  53.0       NaN  53.0  53.0  53.0  53.0  53.0   1.0  1.0  ...   
96         2.0  62.0  2.828427  60.0  61.0  62.0  63.0  64.0   2.0  0.5  ...   
...        ...   ...       ...   ...   ...   ...   ...   ...   ...  ...  ...   
190        1.0  52.0       NaN  52.0  52.0  52.0  52.0  52.0   1.0  1.0  ...   
192        1.0  34.0       NaN  34.0  34.0  34.0  34.0  34.0   1.0  0.0  ...   
194        1.0  42.0       NaN  42.0  42.0  42.0  42.0  42.0   1.0  1.0  ...   
195        1.0  54.0       NaN  54.0  54.0  54.0  54.0  54.0   1.0  1.0  ...   
202        1.0  29.0       NaN  29.0  29.0  29.0  29.0  29.0   1.0  1.0  ...   

         thall      Target                                            
           75%  max  count mean       std  min   25%  50%   75%  max  
thalachh                                                              
71         2.0  2.0    1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
88         3.0  3.0    1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
90         1.0  1.0    1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
95         3.0  3.0    1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
96         2.0  2.0    2.0  0.5  0.707107  0.0  0.25  0.5  0.75  1.0  
...        ...  ...    ...  ...       ...  ...   ...  ...   ...  ...  
190        1.0  1.0    1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0  
192        2.0  2.0    1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0  
194        3.0  3.0    1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0  
195        3.0  3.0    1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
202        2.0  2.0    1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0  

[91 rows x 104 columns]

In [ ]:
dfHeart_discretize.groupby('chol').describe()

age                                                  sex       ...  \
     count  mean        std   min   25%   50%   75%   max count mean  ...   
chol                                                                  ...   
126    1.0  57.0        NaN  57.0  57.0  57.0  57.0  57.0   1.0  1.0  ...   
131    1.0  57.0        NaN  57.0  57.0  57.0  57.0  57.0   1.0  1.0  ...   
141    1.0  44.0        NaN  44.0  44.0  44.0  44.0  44.0   1.0  0.0  ...   
149    2.0  60.0  15.556349  49.0  54.5  60.0  65.5  71.0   2.0  0.5  ...   
157    1.0  41.0        NaN  41.0  41.0  41.0  41.0  41.0   1.0  1.0  ...   
...    ...   ...        ...   ...   ...   ...   ...   ...   ...  ...  ...   
394    1.0  62.0        NaN  62.0  62.0  62.0  62.0  62.0   1.0  0.0  ...   
407    1.0  63.0        NaN  63.0  63.0  63.0  63.0  63.0   1.0  0.0  ...   
409    1.0  56.0        NaN  56.0  56.0  56.0  56.0  56.0   1.0  0.0  ...   
417    1.0  65.0        NaN  65.0  65.0  65.0  65.0  65.0   1.0  0.0  ...   
564    1.0  67.0        NaN  67.0  67.0  67.0  67.0  67.0   1.0  0.0  ...   

     thalachh_interval      oldpeak_interval                                 \
                   75%  max            count mean       std  min   25%  50%   
chol                                                                          
126                3.0  3.0              1.0  0.0       NaN  0.0  0.00  0.0   
131                1.0  1.0              1.0  0.0       NaN  0.0  0.00  0.0   
141                3.0  3.0              1.0  0.0       NaN  0.0  0.00  0.0   
149                2.0  2.0              2.0  0.5  0.707107  0.0  0.25  0.5   
157                4.0  4.0              1.0  0.0       NaN  0.0  0.00  0.0   
...                ...  ...              ...  ...       ...  ...   ...  ...   
394                3.0  3.0              1.0  0.0       NaN  0.0  0.00  0.0   
407                3.0  3.0              1.0  3.0       NaN  3.0  3.00  3.0   
409                3.0  3.0              1.0  1.0       NaN  1.0  1.00  1.0   
417                3.0  3.0              1.0  0.0       NaN  0.0  0.00  0.0   
564                3.0  3.0              1.0  1.0       NaN  1.0  1.00  1.0   

                 
       75%  max  
chol             
126   0.00  0.0  
131   0.00  0.0  
141   0.00  0.0  
149   0.75  1.0  
157   0.00  0.0  
...    ...  ...  
394   0.00  0.0  
407   3.00  3.0  
409   1.00  1.0  
417   0.00  0.0  
564   1.00  1.0  

[152 rows x 104 columns]

In [ ]:
dfHeart_discretize.groupby('caa').describe()

age                                                         sex  \
     count       mean       std   min    25%   50%    75%   max  count   
caa                                                                      
0    175.0  51.645714  9.178321  29.0  44.00  52.0  58.00  76.0  175.0   
1     65.0  57.476923  6.876381  41.0  54.00  57.0  61.00  74.0   65.0   
2     38.0  59.789474  6.576608  42.0  58.00  60.0  64.00  71.0   38.0   
3     20.0  59.900000  8.232797  45.0  52.75  62.0  65.25  77.0   20.0   
4      5.0  45.800000  8.899438  38.0  38.00  43.0  52.00  58.0    5.0   

               ... chol_interval      oldpeak_interval                      \
         mean  ...           75%  max            count      mean       std   
caa            ...                                                           
0    0.634286  ...           1.0  4.0            175.0  0.382857  0.724563   
1    0.769231  ...           1.0  3.0             65.0  0.476923  0.709482   
2    0.657895  ...           1.0  3.0             38.0  0.947368  0.957118   
3    0.800000  ...           2.0  3.0             20.0  1.050000  1.234376   
4    1.000000  ...           1.0  1.0              5.0  0.000000  0.000000   

                              
     min  25%  50%  75%  max  
caa                           
0    0.0  0.0  0.0  1.0  4.0  
1    0.0  0.0  0.0  1.0  2.0  
2    0.0  0.0  1.0  2.0  3.0  
3    0.0  0.0  1.0  1.0  4.0  
4    0.0  0.0  0.0  0.0  0.0  

[5 rows x 112 columns]

In [ ]:
dfHeart_discretize.groupby('thall').describe()

age                                                         sex  \
       count       mean       std   min    25%   50%    75%   max  count   
thall                                                                      
0        2.0  52.500000  0.707107  52.0  52.25  52.5  52.75  53.0    2.0   
1       18.0  56.666667  7.654295  41.0  56.00  57.5  62.00  66.0   18.0   
2      166.0  53.246988  9.856815  29.0  45.00  53.0  61.00  77.0  166.0   
3      117.0  55.632479  7.979066  35.0  51.00  57.0  61.00  70.0  117.0   

                 ... chol_interval      oldpeak_interval                      \
           mean  ...           75%  max            count      mean       std   
thall            ...                                                           
0      0.500000  ...          0.75  1.0              2.0  0.000000  0.000000   
1      0.944444  ...          1.00  2.0             18.0  0.777778  0.878204   
2      0.524096  ...          1.00  3.0            166.0  0.289157  0.572715   
3      0.871795  ...          1.00  4.0            117.0  0.794872  1.004631   

                                
       min  25%  50%  75%  max  
thall                           
0      0.0  0.0  0.0  0.0  0.0  
1      0.0  0.0  1.0  1.0  3.0  
2      0.0  0.0  0.0  0.0  2.0  
3      0.0  0.0  0.0  1.0  4.0  

[4 rows x 112 columns]

In [37]:
dfHeart_discretize.groupby('slp').describe()

age                                                        sex  \
     count       mean       std   min    25%   50%   75%   max  count   
slp                                                                     
0     21.0  55.380952  8.315505  37.0  53.00  56.0  62.0  70.0   21.0   
1    140.0  56.100000  8.437071  35.0  50.75  57.0  62.0  76.0  140.0   
2    142.0  52.507042  9.490381  29.0  44.25  52.0  59.0  77.0  142.0   

               ... chol_interval      oldpeak_interval                      \
         mean  ...           75%  max            count      mean       std   
slp            ...                                                           
0    0.761905  ...           1.0  2.0             21.0  1.666667  1.278019   
1    0.678571  ...           1.0  4.0            140.0  0.700000  0.819721   
2    0.676056  ...           1.0  3.0            142.0  0.154930  0.434272   

                              
     min  25%  50%  75%  max  
slp                           
0    0.0  1.0  2.0  2.0  4.0  
1    0.0  0.0  0.0  1.0  3.0  
2    0.0  0.0  0.0  0.0  3.0  

[3 rows x 112 columns]

In [14]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')
dfHeart_discretize['trtbps_interval'] = pd.cut(dfHeart_discretize['trtbps'], bins=5, labels=False)
dfHeart_discretize['thalachh_interval'] = pd.cut(dfHeart_discretize['thalachh'], bins=5, labels=False)
dfHeart_discretize.drop(columns=["trtbps", "thalachh"], inplace=True)
y = dfHeart_discretize['Target']
X = dfHeart_discretize.drop(columns='Target')

In [3]:
dfHeart_discretize

,age,sex,cp,chol,fbs,restecg,exng,oldpeak,slp,caa,thall,Target,trtbps_interval,thalachh_interval
0,63,1,3,233,1,0,0,2.3,0,0,1,1,2,3
1,37,1,2,250,0,1,0,3.5,0,0,2,1,1,4
2,41,0,1,204,0,0,0,1.4,2,0,2,1,1,3
3,56,1,1,236,0,1,0,0.8,2,0,2,1,1,4
4,57,0,0,354,0,1,1,0.6,2,0,2,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,241,0,1,1,0.2,1,0,3,0,2,1
299,45,1,3,264,0,1,0,1.2,1,0,3,0,0,2
300,68,1,0,193,1,1,0,3.4,1,2,3,0,2,2
301,57,1,0,131,0,1,1,1.2,1,1,3,0,1,1


In [4]:
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X, y, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Média e desvio padrão das métricas após 10 folds:")
print(f"Tempo para treinar o modelo {execution_time_model} segundos")
print(f"Acurácia: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Média e desvio padrão das métricas após 10 folds:
Tempo para treinar o modelo 1.952624797821045 segundos
Acurácia: 0.83 (+/- 0.05)
Recall: 0.88 (+/- 0.08)
F1 Score: 0.85 (+/- 0.04)
Log-loss: 0.40 (+/- 0.09)


In [40]:
from sklearn.ensemble import RandomForestClassifier
from slice_finder import SliceFinder
import warnings
warnings.filterwarnings('ignore')

In [44]:
lr = RandomForestClassifier(max_depth=None, n_estimators=100)
lr.fit(X, y)

sf = SliceFinder(lr, (X, y))
metrics_all, _ = sf.evaluate_model((X,y))
reference = (np.mean(metrics_all), np.std(metrics_all), len(metrics_all))

name = 'Heart_editado.p'
recommendations = sf.find_slice(k=100, epsilon=0.4, degree=2, max_workers=4, name=name)

for s in recommendations:
    print ('\n=====================\nSlice description:')
    for k, v in list(s.filters.items()):
        values = ''
        if k in encoders:
            le = encoders[k]
            for v_ in v:
                values += '%s '%(le.inverse_transform(v_)[0])
        else:
            for v_ in sorted(v, key=lambda x: x[0]):
                if len(v_) > 1:
                    values += '%s ~ %s'%(v_[0], v_[1])
                else:
                    values += '%s '%(v_[0])
        print ('%s:%s'%(k, values))
    print ('---------------------\neffect_size: %s'%(s.effect_size))
    print ('---------------------\nmetric: %s'%(s.metric))
    print ('size: %s'%(s.size))

degree 1
crossing
effect size filtering
degree 2
crossing
effect size filtering
sorting

Slice description:
age:57 
thalachh_interval:3 
---------------------
effect_size: 0.4028389380610816
---------------------
metric: 0.22090329423918723
size: 8

Slice description:
cp:3 
thall:3 
---------------------
effect_size: 0.4568295376441438
---------------------
metric: 0.232838419741222
size: 8

Slice description:
age:57 
slp:2 
---------------------
effect_size: 0.4202791661367745
---------------------
metric: 0.2323519873149651
size: 7

Slice description:
age:57 
trtbps_interval:1 
---------------------
effect_size: 0.43106757476590624
---------------------
metric: 0.24147580239535235
size: 7

Slice description:
age:64 
restecg:1 
---------------------
effect_size: 0.5271798389753682
---------------------
metric: 0.26807202066814984
size: 6

Slice description:
oldpeak:0.2 
thall:3 
---------------------
effect_size: 0.4585510187413712
---------------------
metric: 0.24082457956519418
siz

In [54]:
with open('./resultados/bank/Heart_editado.p', 'rb') as file:
    data = pickle.load(file)
    
    name = []
    aux_1 = []
    aux_2 = []
    
    data = sorted(data, key=lambda d: d.size, reverse=True)
    for d in data:
        print ('\n=====================\nSlice description:')
        for k, v in list(d.filters.items()):
            values = ''
            if k in encoders:
                le = encoders[k]
                for v_ in v:
                    values += '%s '%(le.inverse_transform(v_)[0])
            else:
                for v_ in sorted(v, key=lambda x: x[0]):
                    if len(v_) > 1:
                        values += '%s ~ %s'%(v_[0], v_[1])
                    else:
                        values += '%s '%(v_[0])
            print ('%s:%s'%(k, values))
        print(f'Instâncias da região: {d.data_idx}')
        print(f'Tamanho da região:    {d.size}')
        print(f'Efect size da região: {d.effect_size}')
        print(f'Log Loss da região:    {d.metric}') # Log Loss
        print(f'Tempo da região:      {d.time}')
        print(f'Tempo do predict:    {d.time_modelo}')
        print('\n')
        
        if d.size > 10:
            aux_1.append(d.size)
            aux_2.append(d.effect_size)


Slice description:
age:57 
thalachh_interval:3 
Instâncias da região: Index([4, 9, 59, 91, 111, 210, 287, 302], dtype='int64')
Tamanho da região:    8
Efect size da região: 0.4028389380610816
Log Loss da região:    0.22090329423918723
Tempo da região:      0.020975799998268485
Tempo do predict:    0.00845620000109193



Slice description:
cp:3 
thall:3 
Instâncias da região: Index([24, 83, 101, 117, 152, 228, 259, 299], dtype='int64')
Tamanho da região:    8
Efect size da região: 0.4568295376441438
Log Loss da região:    0.232838419741222
Tempo da região:      0.01924429999780841
Tempo do predict:    0.008574199997383403



Slice description:
age:57 
slp:2 
Instâncias da região: Index([4, 9, 59, 91, 111, 277, 287], dtype='int64')
Tamanho da região:    7
Efect size da região: 0.4202791661367745
Log Loss da região:    0.2323519873149651
Tempo da região:      0.017743500000506174
Tempo do predict:    0.00827910000225529



Slice description:
age:57 
trtbps_interval:1 
Instâncias da regiã

In [56]:
dicionario = {}

for d in data:
    if d.size > 1:
        dicionario[f'{d.filters}'] = []
        for i in d.data_idx:
            dicionario[f'{d.filters}'].append(i)

In [57]:
from collections import Counter
import ast

feature_cont = Counter()

for feature in dicionario.keys():
    key_dict = ast.literal_eval(feature)
    for feature in key_dict.keys():
        feature_cont[feature] += 1

In [58]:
print(feature_cont)

Counter({'age': 29, 'oldpeak': 24, 'cp': 17, 'chol': 15, 'trtbps_interval': 7, 'thall': 6, 'thalachh_interval': 5, 'caa': 5, 'slp': 4, 'sex': 2, 'restecg': 1, 'fbs': 1, 'exng': 1})


In [16]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')
dfHeart_discretize['trtbps_interval'] = pd.cut(dfHeart_discretize['trtbps'], bins=5, labels=False)
dfHeart_discretize['thalachh_interval'] = pd.cut(dfHeart_discretize['thalachh'], bins=5, labels=False)
dfHeart_discretize.drop(columns=["trtbps", "thalachh"], inplace=True)
y = dfHeart_discretize['Target']
X = dfHeart_discretize.drop(columns='Target')

In [17]:
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X, y, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

Mean and standard deviation of metrics after 10 folds:
Time to train the model: 2.8267688751220703 seconds
Accuracy: 0.83 (+/- 0.05)
Recall: 0.88 (+/- 0.08)
F1 Score: 0.85 (+/- 0.04)
Log-loss: 0.40 (+/- 0.09)


In [61]:
dfHeart_discretize.groupby('thalachh_interval').describe()

age                                                       \
                   count       mean       std   min    25%   50%    75%   max   
thalachh_interval                                                               
0                    7.0  60.285714  4.608481  53.0  58.00  60.0  63.00  67.0   
1                   37.0  57.972973  8.590972  40.0  54.00  57.0  64.00  76.0   
2                   88.0  56.818182  8.210654  35.0  51.00  58.0  63.00  71.0   
3                  140.0  53.721429  8.622430  34.0  47.75  54.0  59.25  77.0   
4                   31.0  44.677419  7.533866  29.0  40.00  44.0  49.50  63.0   

                     sex            ... Target      trtbps_interval            \
                   count      mean  ...    75%  max           count      mean   
thalachh_interval                   ...                                         
0                    7.0  0.714286  ...    0.0  1.0             7.0  1.428571   
1                   37.0  0.675676  ...    1.0  1.0            37.0  1.405405   
2                   88.0  0.761364  ...    1.0  1.0            88.0  1.284091   
3                  140.0  0.592857  ...    1.0  1.0           140.0  1.264286   
4                   31.0  0.870968  ...    1.0  1.0            31.0  1.290323   

                                                      
                        std  min  25%  50%  75%  max  
thalachh_interval                                     
0                  0.786796  1.0  1.0  1.0  1.5  3.0  
1                  0.956250  0.0  1.0  1.0  2.0  4.0  
2                  0.946243  0.0  1.0  1.0  2.0  4.0  
3                  0.853353  0.0  1.0  1.0  2.0  4.0  
4                  0.863850  0.0  1.0  1.0  2.0  4.0  

[5 rows x 104 columns]

In [92]:
region = dfHeart_discretize.query('thalachh_interval==0')
region.groupby('Target').describe()

age                                                      sex  \
       count       mean       std   min   25%   50%   75%   max count   
Target                                                                  
0        6.0  60.333333  5.046451  53.0  57.5  60.5  63.5  67.0   6.0   
1        1.0  60.000000       NaN  60.0  60.0  60.0  60.0  60.0   1.0   

                  ... trtbps_interval      thalachh_interval                 \
            mean  ...             75%  max             count mean  std  min   
Target            ...                                                         
0       0.833333  ...            1.75  3.0               6.0  0.0  0.0  0.0   
1       0.000000  ...            1.00  1.0               1.0  0.0  NaN  0.0   

                            
        25%  50%  75%  max  
Target                      
0       0.0  0.0  0.0  0.0  
1       0.0  0.0  0.0  0.0  

[2 rows x 104 columns]

In [96]:
dfHeart_discretize.groupby('trtbps_interval').describe()

age                                                       \
                 count       mean       std   min    25%   50%    75%   max   
trtbps_interval                                                               
0                 52.0  52.038462  8.996648  39.0  44.75  51.0  58.00  71.0   
1                141.0  52.943262  8.965627  29.0  47.00  54.0  59.00  77.0   
2                 84.0  56.059524  8.997123  35.0  50.75  58.0  62.25  76.0   
3                 21.0  61.714286  5.414267  52.0  59.00  59.0  66.00  71.0   
4                  5.0  59.400000  6.228965  54.0  55.00  56.0  64.00  68.0   

                   sex            ... Target      thalachh_interval            \
                 count      mean  ...    75%  max             count      mean   
trtbps_interval                   ...                                           
0                 52.0  0.673077  ...    1.0  1.0              52.0  2.538462   
1                141.0  0.709220  ...    1.0  1.0             141.0  2.489362   
2                 84.0  0.666667  ...    1.0  1.0              84.0  2.583333   
3                 21.0  0.666667  ...    1.0  1.0              21.0  2.095238   
4                  5.0  0.400000  ...    0.0  1.0               5.0  2.600000   

                                                    
                      std  min  25%  50%  75%  max  
trtbps_interval                                     
0                0.827506  1.0  2.0  3.0  3.0  4.0  
1                0.938213  0.0  2.0  3.0  3.0  4.0  
2                0.934072  0.0  2.0  3.0  3.0  4.0  
3                0.830949  0.0  2.0  2.0  3.0  3.0  
4                1.140175  1.0  2.0  3.0  3.0  4.0  

[5 rows x 104 columns]

In [100]:
region = dfHeart_discretize.query('trtbps_interval==3')
region.groupby('Target').describe()

age                                                        sex  \
       count       mean       std   min    25%   50%    75%   max count   
Target                                                                    
0       13.0  61.230769  4.548880  55.0  59.00  59.0  66.00  70.0  13.0   
1        8.0  62.500000  6.866066  52.0  57.75  64.5  66.75  71.0   8.0   

                  ... trtbps_interval      thalachh_interval            \
            mean  ...             75%  max             count      mean   
Target            ...                                                    
0       0.692308  ...             3.0  3.0              13.0  1.769231   
1       0.625000  ...             3.0  3.0               8.0  2.625000   

                                           
             std  min  25%  50%  75%  max  
Target                                     
0       0.832050  0.0  1.0  2.0  2.0  3.0  
1       0.517549  2.0  2.0  3.0  3.0  3.0  

[2 rows x 104 columns]

# Apply SMOTE

In [18]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')
dfHeart_discretize['trtbps_interval'] = pd.cut(dfHeart_discretize['trtbps'], bins=5, labels=False)
dfHeart_discretize['thalachh_interval'] = pd.cut(dfHeart_discretize['thalachh'], bins=5, labels=False)
dfHeart_discretize.drop(columns=["trtbps", "thalachh"], inplace=True)
y = dfHeart_discretize['Target']
X = dfHeart_discretize.drop(columns='Target')

In [19]:
region = ['trtbps_interval==3']

slice_remove = []
slice_remove_idx = []
X_slice = []
y_slice = []
region_SMOTE = []

smote = SMOTE(sampling_strategy=0.9, random_state=42)

for region_condition in region:
    slice_remove.append(dfHeart_discretize.query(region_condition))
    slice_remove_idx.append(dfHeart_discretize.query(region_condition).index)

for i in range(len(region)):
    dfHeart_discretize.drop(slice_remove_idx[i], inplace=True)
    y_slice.append(slice_remove[i]['Target'])
    X_slice.append(slice_remove[i].drop(columns='Target'))
    
    X_slice_smote, y_slice_smote = smote.fit_resample(X_slice[i], y_slice[i])
    region_SMOTE.append(pd.concat([pd.DataFrame(X_slice_smote, columns=X_slice[i].columns), pd.Series(y_slice_smote, name='Target')], axis=1))
    
df_SMOTE_slice = pd.concat(region_SMOTE + [dfHeart_discretize], ignore_index=True)
X_SMOTE_data = df_SMOTE_slice.drop(columns='Target')
y_SMOTE_data = df_SMOTE_slice['Target']
len(df_SMOTE_slice)

306

In [20]:
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print(len(X_SMOTE_data))
print("Mean and standard deviation of metrics after 10 folds:")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

306
Mean and standard deviation of metrics after 10 folds:
Time to train the model: 4.417855978012085 seconds
Accuracy: 0.84 (+/- 0.06)
Recall: 0.89 (+/- 0.04)
F1 Score: 0.86 (+/- 0.05)
Log-loss: 0.37 (+/- 0.09)


In [119]:
dfHeart_discretize.groupby('cp').describe()

age                                                         sex  \
    count       mean        std   min   25%   50%    75%   max  count   
cp                                                                      
0   133.0  55.308271   8.426312  35.0  50.0  57.0  61.00  77.0  133.0   
1    48.0  50.645833   9.002339  29.0  44.0  52.0  56.25  74.0   48.0   
2    83.0  53.192771   9.381427  37.0  46.0  53.0  60.00  76.0   83.0   
3    18.0  54.166667  10.556570  34.0  46.5  57.0  62.50  69.0   18.0   

              ... trtbps_interval      thalachh_interval                      \
        mean  ...             75%  max             count      mean       std   
cp            ...                                                              
0   0.736842  ...             2.0  4.0             133.0  2.195489  0.941136   
1   0.645833  ...             1.0  4.0              48.0  2.979167  0.699227   
2   0.602410  ...             2.0  4.0              83.0  2.710843  0.804410   
3   0.777778  ...             2.0  2.0              18.0  2.944444  0.872604   

                               
    min   25%  50%   75%  max  
cp                             
0   0.0  2.00  2.0  3.00  4.0  
1   1.0  3.00  3.0  3.00  4.0  
2   0.0  2.00  3.0  3.00  4.0  
3   1.0  2.25  3.0  3.75  4.0  

[4 rows x 104 columns]

In [138]:
region = dfHeart_discretize.query('cp==1')
region.groupby('Target').describe()

age                                                      sex  \
       count       mean       std   min   25%   50%   75%   max count   
Target                                                                  
0        9.0  57.444444  4.952553  48.0  57.0  57.0  58.0  66.0   9.0   
1       41.0  50.024390  9.786439  29.0  44.0  49.0  56.0  74.0  41.0   

                  ... trtbps_interval      thalachh_interval            \
            mean  ...             75%  max             count      mean   
Target            ...                                                    
0       0.777778  ...             2.0  4.0               9.0  2.555556   
1       0.609756  ...             1.0  3.0              41.0  3.024390   

                                           
             std  min  25%  50%  75%  max  
Target                                     
0       1.013794  1.0  2.0  3.0  3.0  4.0  
1       0.651452  1.0  3.0  3.0  3.0  4.0  

[2 rows x 104 columns]

In [21]:
region = ['cp==1']

slice_remove = []
slice_remove_idx = []
X_slice = []
y_slice = []
region_SMOTE = []

smote = SMOTE(sampling_strategy=0.9, random_state=42)

for region_condition in region:
    slice_remove.append(dfHeart_discretize.query(region_condition))
    slice_remove_idx.append(dfHeart_discretize.query(region_condition).index)

for i in range(len(region)):
    dfHeart_discretize.drop(slice_remove_idx[i], inplace=True)
    y_slice.append(slice_remove[i]['Target'])
    X_slice.append(slice_remove[i].drop(columns='Target'))
    
    X_slice_smote, y_slice_smote = smote.fit_resample(X_slice[i], y_slice[i])
    region_SMOTE.append(pd.concat([pd.DataFrame(X_slice_smote, columns=X_slice[i].columns), pd.Series(y_slice_smote, name='Target')], axis=1))
    
df_SMOTE_slice = pd.concat(region_SMOTE + [dfHeart_discretize], ignore_index=True)
X_SMOTE_data = df_SMOTE_slice.drop(columns='Target')
y_SMOTE_data = df_SMOTE_slice['Target']
len(df_SMOTE_slice)

310

In [22]:
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

start_time_model = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

print(len(X_SMOTE_data))
print("Mean and standard deviation of metrics after 10 folds:")
print(f"Time to train the model: {execution_time_model} seconds")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")

310
Mean and standard deviation of metrics after 10 folds:
Time to train the model: 5.340811014175415 seconds
Accuracy: 0.84 (+/- 0.05)
Recall: 0.84 (+/- 0.09)
F1 Score: 0.84 (+/- 0.05)
Log-loss: 0.40 (+/- 0.05)


# Random remove

In [23]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

df_temp = dfHeart_discretize.copy()
remove_rows = df_temp.sample(n=14, random_state=42).index
print(remove_rows)

df_temp.drop(remove_rows, inplace=True)  
y_temp = df_temp['Target']
X_temp = df_temp.drop(columns=['Target'])

scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results_all = cross_validate(rf, X_temp, y_temp, cv=skf, scoring=scoring)
    
print("Mean and standard deviation of metrics after 10 folds:")
print(f"Dataset size after removal: {len(df_temp)}")
print(f"Accuracy: {np.mean(results_all['test_accuracy']):.2f} (+/- {np.std(results_all['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_all['test_recall']):.2f} (+/- {np.std(results_all['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_all['test_f1']):.2f} (+/- {np.std(results_all['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_all['test_log_loss']):.2f} (+/- {np.std(results_all['test_log_loss']):.2f})")


Index([179, 228, 111, 246, 60, 9, 119, 223, 268, 33, 5, 101, 45, 175], dtype='int64')
Mean and standard deviation of metrics after 10 folds:
Dataset size after removal: 289
Accuracy: 0.82 (+/- 0.04)
Recall: 0.85 (+/- 0.09)
F1 Score: 0.84 (+/- 0.04)
Log-loss: 0.52 (+/- 0.38)


In [24]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = dfHeart_discretize.query('cp==3 and sex==1').index
count = len(dfHeart_discretize)

metrics_summary = {
    "accuracy": [],
    "recall": [],
    "f1": [],
    "log_loss": [],
    "execution_time": []
}

for i in range(10):
    
    df_temp = dfHeart_discretize.copy()
    remove_rows = df_temp.sample(n=len(region), random_state=i*7).index
    df_temp.drop(remove_rows, inplace=True)
    
    y_temp = df_temp['Target']
    X_temp = df_temp.drop(columns=['Target'])

    scoring = {
            'accuracy': 'accuracy',  
            'recall': 'recall',      
            'f1': 'f1',              
            'log_loss': make_scorer(log_loss, response_method='predict_proba')
        }

    rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    results_all = cross_validate(rf, X_temp, y_temp, cv=skf, scoring=scoring)
    
    metrics_summary["accuracy"].append(np.mean(results_all['test_accuracy']))
    metrics_summary["recall"].append(np.mean(results_all['test_recall']))
    metrics_summary["f1"].append(np.mean(results_all['test_f1']))
    metrics_summary["log_loss"].append(np.mean(results_all['test_log_loss']))

final_results = {
    "accuracy_mean": np.mean(metrics_summary["accuracy"]),
    "accuracy_std": np.std(metrics_summary["accuracy"]),
    "recall_mean": np.mean(metrics_summary["recall"]),
    "recall_std": np.std(metrics_summary["recall"]),
    "f1_mean": np.mean(metrics_summary["f1"]),
    "f1_std": np.std(metrics_summary["f1"]),
    "log_loss_mean": np.mean(metrics_summary["log_loss"]),
    "log_loss_std": np.std(metrics_summary["log_loss"]),
}

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Accuracy: {final_results['accuracy_mean']:.2f} (+/- {final_results['accuracy_std']:.2f})")
print(f"Recall: {final_results['recall_mean']:.2f} (+/- {final_results['recall_std']:.2f})")
print(f"F1 Score: {final_results['f1_mean']:.2f} (+/- {final_results['f1_std']:.2f})")
print(f"Log-loss: {final_results['log_loss_mean']:.2f} (+/- {final_results['log_loss_std']:.2f})")

Mean and standard deviation of metrics after 10 folds:
Accuracy: 0.83 (+/- 0.01)
Recall: 0.87 (+/- 0.01)
F1 Score: 0.84 (+/- 0.01)
Log-loss: 0.40 (+/- 0.01)


In [25]:
import pandas as pd
from utils import *
from utils_datasets import *

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = dfHeart_discretize.query('cp==3 and sex==1').index
count = len(dfHeart_discretize)

metrics_summary = {
    "accuracy": [],
    "recall": [],
    "f1": [],
    "log_loss": [],
    "execution_time": []
}

for i in range(10):
    df_temp = dfHeart_discretize.copy()
    
    remove_rows = df_temp.sample(n=len(region), random_state=i*3).index
    df_temp.drop(remove_rows, inplace=True)
    
    y_temp = df_temp['Target']
    X_temp = df_temp.drop(columns=['Target'])

    scoring = {
        'accuracy': 'accuracy',  
        'recall': 'recall',      
        'f1': 'f1',              
        'log_loss': make_scorer(log_loss, response_method='predict_proba')
    }

    
    rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    start_time_model = time.time()
    results_all = cross_validate(rf, X, y, cv=skf, scoring=scoring)
    end_time_model = time.time()
    execution_time_model = end_time_model - start_time_model


    metrics_summary["accuracy"].append(np.mean(results_all['test_accuracy']))
    metrics_summary["recall"].append(np.mean(results_all['test_recall']))
    metrics_summary["f1"].append(np.mean(results_all['test_f1']))
    metrics_summary["log_loss"].append(np.mean(results_all['test_log_loss']))
    metrics_summary["execution_time"].append(execution_time_model)

final_results = {
    "accuracy_mean": np.mean(metrics_summary["accuracy"]),
    "accuracy_std": np.std(metrics_summary["accuracy"]),
    "recall_mean": np.mean(metrics_summary["recall"]),
    "recall_std": np.std(metrics_summary["recall"]),
    "f1_mean": np.mean(metrics_summary["f1"]),
    "f1_std": np.std(metrics_summary["f1"]),
    "log_loss_mean": np.mean(metrics_summary["log_loss"]),
    "log_loss_std": np.std(metrics_summary["log_loss"]),
}

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Accuracy: {final_results['accuracy_mean']:.2f} (+/- {final_results['accuracy_std']:.2f})")
print(f"Recall: {final_results['recall_mean']:.2f} (+/- {final_results['recall_std']:.2f})")
print(f"F1 Score: {final_results['f1_mean']:.2f} (+/- {final_results['f1_std']:.2f})")
print(f"Log-loss: {final_results['log_loss_mean']:.2f} (+/- {final_results['log_loss_std']:.2f})")

Mean and standard deviation of metrics after 10 folds:
Accuracy: 0.81 (+/- 0.00)
Recall: 0.84 (+/- 0.00)
F1 Score: 0.82 (+/- 0.00)
Log-loss: 0.41 (+/- 0.00)


In [26]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import log_loss, make_scorer
import time

df_heart = import_datasets('heart')
dfHeart_discretize, X, y, encoders = encoder(df_heart, 'heart')

region = dfHeart_discretize.query('cp==3 and sex==1').index
count = len(dfHeart_discretize)

# Armazenar métricas
metrics_summary = {
    "accuracy": [],
    "recall": [],
    "f1": [],
    "log_loss": [],
    "execution_time": []
}

for i in range(5):
    df_temp = dfHeart_discretize.copy()
    
    remove_rows = df_temp.sample(n=len(region), random_state=i).index
    df_temp.drop(remove_rows, inplace=True)
    
    y_temp = df_temp['Target']
    X_temp = df_temp.drop(columns=['Target'])

    scoring = {
        'accuracy': 'accuracy',
        'recall': 'recall',
        'f1': 'f1',
        'log_loss': make_scorer(log_loss, response_method='predict_proba')
    }

    rf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    start_time_model = time.time()
    results_all = cross_validate(rf, X_temp, y_temp, cv=skf, scoring=scoring)
    end_time_model = time.time()
    execution_time_model = end_time_model - start_time_model

    metrics_summary["accuracy"].append(np.mean(results_all['test_accuracy']))
    metrics_summary["recall"].append(np.mean(results_all['test_recall']))
    metrics_summary["f1"].append(np.mean(results_all['test_f1']))
    metrics_summary["log_loss"].append(np.mean(results_all['test_log_loss']))
    metrics_summary["execution_time"].append(execution_time_model)

final_results = {
    "accuracy_mean": np.mean(metrics_summary["accuracy"]),
    "accuracy_std": np.std(metrics_summary["accuracy"]),
    "recall_mean": np.mean(metrics_summary["recall"]),
    "recall_std": np.std(metrics_summary["recall"]),
    "f1_mean": np.mean(metrics_summary["f1"]),
    "f1_std": np.std(metrics_summary["f1"]),
    "log_loss_mean": np.mean(metrics_summary["log_loss"]),
    "log_loss_std": np.std(metrics_summary["log_loss"]),
    "execution_time_mean": np.mean(metrics_summary["execution_time"]),
    "execution_time_std": np.std(metrics_summary["execution_time"]),
}

print("Mean and standard deviation of metrics after 10 folds:")
print(f"Accuracy: {final_results['accuracy_mean']:.2f} (+/- {final_results['accuracy_std']:.2f})")
print(f"Recall: {final_results['recall_mean']:.2f} (+/- {final_results['recall_std']:.2f})")
print(f"F1 Score: {final_results['f1_mean']:.2f} (+/- {final_results['f1_std']:.2f})")
print(f"Log-loss: {final_results['log_loss_mean']:.2f} (+/- {final_results['log_loss_std']:.2f})")

Mean and standard deviation of metrics after 10 folds:
Accuracy: 0.82 (+/- 0.02)
Recall: 0.86 (+/- 0.01)
F1 Score: 0.84 (+/- 0.01)
Log-loss: 0.43 (+/- 0.01)
